# Collection `Activities`:
- _id : Mã định danh tự động từ Mongodb
- Title : Tên hoạt động
- Url : Đường dẫn đến hoạt động đó
- Img : Hình ảnh minh hoạ cho hoạt động (dưới dạng src)
- Hashtag : Điểm nổi bật của hoạt động (Chiến dịch và khuyến mãi theo mùa) (Có thể None)
- Position : Vị trí diễn ra hoạt động 
- Rating : Chỉ số rating trung bình và lượt đánh giá
- Orig_Price : Giá gốc (Có thể None)
- Discount_Price : Giá sau khi giảm 
- Datetime : Ngày giờ thu thập
- Source : Nguồn thu thập dữ liệu

# Library

In [1]:
import pandas as pd
import time
import random
import numpy as np
from datetime import datetime
import json

import requests
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException

from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger
from apscheduler.jobstores.mongodb import MongoDBJobStore
from bson import ObjectId as BsonObjectId
from db_traveloka import Province_URL_Activities
from db_traveloka import Activities_Reviews

from Details_Scraping_Modules import Scraping_Details

# Hàm Create_WebDriver
Khởi tạo WebDriver

In [2]:
def Create_WebDriver ():  
    """Khởi tạo WebDriver với cấu hình tùy chỉnh"""  
    options = Options()
    # Bỏ comment nếu muốn chạy headless (ẩn trình duyệt)
    # options.add_argument('--headless')

    # Các option giúp chạy ổn định trên macOS
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    # Giúp ẩn dấu hiệu tự động hóa
    options.add_argument('--disable-blink-features=AutomationControlled')

    # Tắt popup, extension
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-popup-blocking')

    # Giả lập user-agent trình duyệt thật
    options.add_argument(
        "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    )

    # Tắt GPU cho máy không dùng đồ họa hoặc headless
    options.add_argument('--disable-gpu')

    # Cỡ cửa sổ trình duyệt
    options.add_argument('--window-size=1920,1080')

    driver = webdriver.Chrome(options=options)

    # Ẩn navigator.webdriver giúp tránh bị phát hiện bot
    driver.execute_cdp_cmd(
        'Page.addScriptToEvaluateOnNewDocument',
        {
            'source': '''
                Object.defineProperty(navigator, 'webdriver', {get: () => undefined})
            '''
        }
    )

    # Ví dụ random delay mô phỏng người dùng
    time.sleep(random.uniform(2,5))
    return driver

# Hàm Process_Price
Xử lý trường Price: Tách thành 2 trường: Original_Price và Final_Price

In [3]:
def Process_Price(price_str):
    """Xử lý giá trị Price"""
    # Loại bỏ ký tự 'VND' và khoảng trắng
    price_str = price_str.replace('VND', '').strip()
    
    # Tách chuỗi thành list các giá
    prices = [p.strip() for p in price_str.split() if p.strip()]
    
    # Khởi tạo dictionary kết quả
    result = {'Original_Price': None, 'Final_Price': None}
    
    # Xử lý trường hợp
    if len(prices) == 2:
        result['Original_Price'] = prices[0]
        result['Final_Price'] = prices[1]
    elif len(prices) == 1:
        result['Final_Price'] = prices[0]
    
    return result

# Dictionary Address Processing

In [4]:
with open('/Users/kittnguyen/Library/Mobile Documents/com~apple~CloudDocs/Tiền xử lý và xây dựng bộ dữ liệu/Thực hành/BTTH5/TPTTTW_quan.json', encoding='utf-8') as f:
    tp_quan = json.load(f)
thu_duc_quan_cu = ['thủ đức', '2', '9']

# Bổ sung vào danh sách nếu chưa có
for q in thu_duc_quan_cu:
    if q not in tp_quan['hồ chí minh']:
        tp_quan['hồ chí minh'].append(q)

In [5]:
with open('/Users/kittnguyen/Library/Mobile Documents/com~apple~CloudDocs/Tiền xử lý và xây dựng bộ dữ liệu/Thực hành/BTTH5/city_ward.json', encoding='utf-8') as f:
    city_ward = json.load(f)

In [6]:
with open('/Users/kittnguyen/Library/Mobile Documents/com~apple~CloudDocs/Tiền xử lý và xây dựng bộ dữ liệu/Thực hành/BTTH5/tinh_tpthuoctinh.json', encoding='utf-8') as f:
    tinh_tp = json.load(f)

In [35]:

with open('/Users/kittnguyen/Library/Mobile Documents/com~apple~CloudDocs/Tiền xử lý và xây dựng bộ dữ liệu/Thực hành/BTTH5/tinh_dis.json', encoding='utf-8') as f:
    tinh_dis = json.load(f)
tinh_dis

{'an giang': ['an phú',
  'tân châu',
  'phú tân',
  'châu phú',
  'tịnh biên',
  'tri tôn',
  'châu thành',
  'chợ mới',
  'thoại sơn'],
 'bà rịa - vũng tàu': ['châu đức',
  'xuyên mộc',
  'long điền',
  'đất đỏ',
  'phú mỹ'],
 'bình dương': ['bàu bàng', 'dầu tiếng', 'phú giáo', 'bắc tân uyên'],
 'bình phước': ['phước long',
  'bình long',
  'bù gia mập',
  'lộc ninh',
  'bù đốp',
  'hớn quản',
  'đồng phú',
  'bù đăng',
  'chơn thành',
  'phú riềng'],
 'bình thuận': ['la gi',
  'tuy phong',
  'bắc bình',
  'hàm thuận bắc',
  'hàm thuận nam',
  'tánh linh',
  'đức linh',
  'hàm tân',
  'phú quí'],
 'bình định': ['an lão',
  'hoài nhơn',
  'hoài ân',
  'phù mỹ',
  'vĩnh thạnh',
  'tây sơn',
  'phù cát',
  'an nhơn',
  'tuy phước',
  'vân canh'],
 'bạc liêu': ['hồng dân',
  'phước long',
  'vĩnh lợi',
  'giá rai',
  'đông hải',
  'hoà bình'],
 'bắc giang': ['yên thế',
  'tân yên',
  'lạng giang',
  'lục nam',
  'lục ngạn',
  'sơn động',
  'yên dũng',
  'việt yên',
  'hiệp hòa'],
 'bắc k

# Hàm Process_Position
Xử lý trường Position

In [8]:
def district_city (place):
    place = place.replace ('quận ','')
    for city, district in tp_quan.items ():
        if place in district:
            return city

def Process_Position(position):
    # Danh sách thành phố trực thuộc trung ương
    central_cities = ["hà nội", "hồ chí minh", "đà nẵng", "hải phòng", "cần thơ"]
    position = position.lower ()
    # Tách Position thành hai phần
    parts = [part.strip() for part in position.split(',')]
    if len(parts) != 2:
        second_part = parts[0]
    else:
        second_part = parts[1]
    
    if 'quận' in second_part:
        second_part = district_city (second_part)
    
    if 'xã' in second_part or 'phường' in second_part or 'thị trấn' in second_part:
        second_part = second_part.replace('xã ', '').replace('phường ', '').replace ('thị trấn ', '')
        for city, ward in city_ward.items ():
            if second_part in ward:
                second_part = city
                
    # for city, ward in city_ward.items ():
    #     if second_part in ward:
    #         second_part = city

    if 'huyện' in second_part or 'thị xã' in second_part or 'thành phố' in second_part:
        second_part = second_part.replace('huyện ', '').replace('thị xã ', '').replace ('thành phố ', '')
    
    # Trường hợp 1: Địa danh thứ hai là thành phố trực thuộc trung ương
    if second_part in central_cities:
        return [second_part]
    
    # Trường hợp 2: Kiểm tra trong dictionary tỉnh và thành phố trực thuộc
    # Nếu địa danh thứ hai là key (tỉnh)
    if second_part in tinh_tp:
        return [second_part]

    # Nếu địa danh thứ hai là value (thành phố trực thuộc tỉnh)
    for province, cities in tinh_tp.items():
        if second_part in cities:
            return [second_part, province]
        
    for province, dis in tinh_dis.items ():
        if second_part in dis:
            return [province]
    # Trường hợp 3: Không thỏa điều kiện
    return None

# Hàm Extract_Info_From_Soup
Thu thập thông tin trong từng trang

In [9]:
def random_delay(min_sec=2, max_sec=5):
    '''Gọi random_delay sau mỗi trang scrape'''
    time.sleep(random.uniform(min_sec, max_sec))

In [10]:
def Scraping_Details_In_One_Page (soup):
    details = {
        'Location' : None,
        'TimeWorking' : None,
        'Description' : None
    }
    your_experience = soup.select ('div[class="css-1dbjc4n r-1ow6zhx r-88pszg"]')
    
    # Scrape Địa điểm và Giờ hoạt động chi tiết
    locat_time_tag = soup.select_one ('div[class="css-1dbjc4n r-eqz5dr"]')
    location_tag = locat_time_tag.select_one ('div[class="css-1dbjc4n r-1awozwy r-18u37iz r-zmhzs6"]')
    location = location_tag.get_text (separator=' ', strip=True) if location_tag else None
    time_work_tag = locat_time_tag.select_one ('div[class="css-1dbjc4n r-eqz5dr"] > div[class="css-1dbjc4n r-1awozwy r-18u37iz r-1h0z5md"] > div[class="css-1dbjc4n r-1awozwy r-18u37iz"]')
    time_work = time_work_tag.get_text (separator=' ', strip=True) if time_work_tag else None
    details ['Location'] = location
    details ['TimeWorking'] = time_work
    descrip = []
    for line in your_experience:
        img_tag = line.select_one ('img[importance="low"]')
        img_src = img_tag['src'] if img_tag else None
        text = line.get_text(separator=' ', strip=True)

        # Skip nếu không có ảnh và không có text
        if not img_src and not text:
            continue
        if text:
            descrip.append (text)
        if img_src:
            descrip.append (img_src)
    details ['Description'] = descrip
    random_delay ()
    return details

# A

In [11]:
def get_metadata(source):
    now = datetime.now()
    metadata = {
        "Source": source,
        "Date": now.strftime("%Y-%m-%d"),       # ví dụ: '2025-06-11'
        "Time": now.strftime("%H:%M:%S")        # ví dụ: '17:36:27'
    }
    return metadata

def safe_find_text(parent, xpath):
    try:
        el = parent.find_element(By.XPATH, xpath)
        return el.text.strip().replace('\n', ' ')
    except:
        return None
    
def safe_find_attr(parent, xpath, attr):
    try:
        el = parent.find_element(By.XPATH, xpath)
        return el.get_attribute(attr)
    except:
        return None
    
def Extract_Info_From_Soup (driver, source, search_city):
    metadata = get_metadata(source)
    data = driver.find_element(By.XPATH, '//div[@class="css-1dbjc4n r-zo7nv5 r-6413gk r-1d71kkm r-c8eef1 r-3mtglp"]')
    tags = driver.find_elements(By.XPATH, '//div[@class="css-1dbjc4n r-1pi2tsx r-13qz1uu"]')

    k = 1
    for tag in tags:
        info = {
            'Title': None,
            'Img': None,
            'Hashtag': None,
            'Position': None,
            'Related_City/Province': None,
            'Rating': None,
            'Original_Price': None,
            'Final_Price': None,
            'Search': search_city
        }

        up_tag = tag.find_element(By.XPATH, './/div[@class="css-1dbjc4n r-p9fnmn"]')
        if up_tag:
            info['Img'] = safe_find_attr(up_tag, './/img[@importance="low"]', 'srcset')
            info['Hashtag'] = up_tag.text.strip().replace('\n', ' ') if up_tag.text else None

        down_tag = tag.find_element(By.XPATH, './/div[@class="css-1dbjc4n r-16y2uox r-1wbh5a2 r-1wtj0ep r-1e081e0 r-1yzf0co"]')
        if down_tag:
            pos = safe_find_text(down_tag, './/div[@class="css-1dbjc4n r-1awozwy r-18u37iz r-1h0z5md"]')
            info['Position'] = pos
            info['Related_City/Province'] = Process_Position(pos)

            title = safe_find_text(down_tag, './/div[@class="css-1dbjc4n"]//h2')
            info['Title'] = title

            downer_tag = down_tag.find_element(By.XPATH, './/div[@class="css-1dbjc4n r-16y2uox r-1wbh5a2 r-17s6mgv r-5njf8e"]')
            if downer_tag:
                info['Rating'] = safe_find_text(downer_tag, './/div[@class="css-1dbjc4n r-1awozwy r-6koalj r-18u37iz"]')

                price_text = safe_find_text(downer_tag, './/div[@class="css-1dbjc4n r-obd0qt r-16y2uox r-1cmwbt1 r-1pi2tsx r-17s6mgv"]')
                price = Process_Price(price_text)
                if price:
                    info.update(price)

        print ('Xử lý bài đăng thứ', k)
        max_retries = 3
        attempt = 0
        success = False
        while attempt < max_retries and not success:
            try:
                # Scroll trang web đến khi gặp phần tử div
                driver.execute_script("arguments[0].scrollIntoView(true);", tag)
                time.sleep(2)
                # Lưu ID của tab hiện tại (cửa sổ chính)
                main_window = driver.current_window_handle
                tag.click()
                
                # Chuyển sang tab mới
                WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
                for handle in driver.window_handles:
                    if handle != main_window:
                        driver.switch_to.window(handle)
                        break
                    
                soup = BeautifulSoup (driver.page_source, 'html.parser')
                full_details = Scraping_Details_In_One_Page (soup)
                current_url = driver.current_url
                info ['Url'] = current_url

                # Đóng tab mới và quay về
                driver.close()
                driver.switch_to.window(main_window)

                random_delay ()
                
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//div[@class="css-1dbjc4n r-1pi2tsx r-13qz1uu"]'))
                )

                if full_details:
                    info.update(full_details)
                    success = True  # Thành công, thoát vòng lặp

            except Exception as e:
                attempt += 1
                print(f"Lỗi khi xử lý bài đăng (lần thử {attempt}):", e)
                if attempt == max_retries:
                    print(f"Bỏ qua bài đăng {title} sau 3 lần thử.")
                    
        if (k + 1) % 5 == 0:
            delay = random.uniform(5, 8)
            print(f"😴 Sleeping longer for {delay:.2f}s...")
            time.sleep(delay)
        else:
                random_delay(2, 5)
        
        # Cập nhật metadata
        if metadata:
            info.update(metadata)
        k+=1
        Province_URL_Activities.insert_one (info)

# Hàm Scraping Pages
Scraping tất cả các trang đối với 1 URL

In [12]:
def Scraping_Pages (driver, source, place):
    wait = WebDriverWait(driver, 15)
    page_count = 1
    while (True):
        print(f"🔍 Đang scrape trang {page_count}...")

        try:
            # 🟡 Đợi cho phần nội dung chính hiển thị
            wait.until(EC.presence_of_element_located(
                (By.XPATH, '//div[@class="css-1dbjc4n r-zo7nv5 r-6413gk r-1d71kkm r-c8eef1 r-3mtglp"]')
            ))
        except TimeoutException:
            print("⚠️ Không tìm thấy nội dung chính, bỏ qua trang này.")
            break

        # ✅ Trang đã load đủ, giờ mới scrape
        Extract_Info_From_Soup(driver, source, place)
        try:
            # Tìm nút "Next"
            next_button = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//div[@data-testid="search_next_page"]')
            ))

            # Scroll tới nút đó để đảm bảo nó hiển thị
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            time.sleep(2)

            # Đợi cho tới khi nút thật sự có thể click
            next_button = wait.until(EC.element_to_be_clickable(
                (By.XPATH, '//div[@data-testid="search_next_page"]')
            ))

            try:
                next_button.click()
            except ElementClickInterceptedException:
                print("⚠️ Nút bị chặn, dùng JavaScript để click.")
                driver.execute_script("arguments[0].click();", next_button)

            # Đợi trang mới load xong hoàn toàn
            wait.until(lambda d: d.execute_script("return document.readyState") == "complete")

            # Đợi phần tử chính của nội dung mới xuất hiện (chống lỗi trắng trang)
            wait.until(EC.presence_of_element_located(
                (By.XPATH, '//div[@class="css-1dbjc4n r-zo7nv5 r-6413gk r-1d71kkm r-c8eef1 r-3mtglp"]')
            ))

            # Delay ngẫu nhiên
            random_delay()  # Giảm thời gian delay để tối ưu
            
            print(f"✅ Đã scrape xong trang {page_count}")
            page_count += 1

        except (NoSuchElementException, TimeoutException):
            print("✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.")
            break
        except StaleElementReferenceException:
            print("⚠️ Nút Next bị stale, thử lại.")
            continue
        except Exception as e:
            print(f"❌ Lỗi khi xử lý trang {page_count + 1}: {e}")
            break

# Hàm Scraping_All_URL
Scraping trên tất cả URL

In [13]:
links = [
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009794', 'Hồ Chí Minh'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009866', 'Cần Thơ'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009843', 'Hà Nội'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010083', 'Đà Nẵng'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009800', 'Hải Phòng'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010311', 'Huế'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010379', 'Lai Châu'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010002', 'Lào Cai'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010647', 'Lào Cai'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010592', 'Hà Giang'],
    # Cao Bằng
    # Lạng Sơn
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010700', 'Yên Bái'],
    # Tuyên Quang
    # Bắc Kạn
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009879', 'Thái Nguyên'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009798', 'Sơn La'],
    # Phú Thọ
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009875', 'Vĩnh Phúc'],
    # Bắc Ninh
    # Bắc Giang
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010278', 'Hạ Long - Quảng Ninh'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009919', 'Hoà Bình'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009834', 'Văn Giang - Hưng Yên'],
    # Hải Dương
    # Thái Bình
    # Hà Nam
    # Nam Định
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009897', 'Ninh Bình'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009808', 'Thanh Hoá'],
    # Nghệ An
    # Hà Tĩnh
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009827', 'Quảng Bình'],
    # Quảng Trị
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009805', 'Quảng Nam'],
    # Quảng Ngãi
    # Kon Tum
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009960', 'Gia Lai'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009974', 'Bình Định'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009823', 'Phú Yên'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010050', 'Đắk Lắk'],
    # Đắk Nông
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010067', 'Cam Ranh - Khánh Hoà'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009841', 'Nha Trang - Khánh Hoà'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010000', 'Lâm Đồng'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009796', 'Ninh Thuận'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009825', 'Bình Thuận'],
    # Bình Phước
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009931', 'Tây Ninh'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010019', 'Bình Dương'],
    # Đồng Nai
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009889', 'Bà Rịa - Vũng Tàu'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010439', 'Long An'],
    # Đồng Tháp
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010107', 'Mỹ Tho - Tiền Giang'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010020', 'Bến Tre'],
    # An Giang
    # Vĩnh Long
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10011570', 'Phú Quốc - Kiên Giang']
    # Hậu Giang
    # Trà Vinh
    # Sóc Trăng
    # Bạc Liêu
    # Cà Mau
]

In [14]:
def Scraping_All_URL (links):
    driver = Create_WebDriver ()
    try:
        for link in links:
            place = link [1]
            source = link [0]
            retries = 0
            success = False
            while retries < 3 and not success:
                try:
                    print(f"Đang xử lý: {place} ({source}) - Lần thử {retries + 1}")
                    driver.get(source)
                    
                    Scraping_Pages(driver, source, place)
                    time.sleep (2)
                    random_delay()
                    success = True  # Nếu không lỗi thì thoát retry
                except Exception as e:
                    retries += 1
                    print(f"Lỗi với {place} ({source}) - Lần {retries}: {e}")
                    random_delay (3,6)
                    if retries == 3:
                        print(f"Bỏ qua {place} sau 3 lần thử.")
                        random_delay (2,5)
            delay = random.uniform(5, 8)
            print(f"😴 Sleeping longer for {delay:.2f}s...")
            time.sleep(delay)
    finally:
        driver.quit()

In [444]:
links = [
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010002', 'Lào Cai'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010647', 'Lào Cai'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010592', 'Hà Giang'],
    # Cao Bằng
    # Lạng Sơn
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010700', 'Yên Bái'],
    # Tuyên Quang
    # Bắc Kạn
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009879', 'Thái Nguyên'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009798', 'Sơn La'],
    # Phú Thọ
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009875', 'Vĩnh Phúc'],
    # Bắc Ninh
    # Bắc Giang
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010278', 'Hạ Long - Quảng Ninh'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009919', 'Hoà Bình'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009834', 'Văn Giang - Hưng Yên'],
    # Hải Dương
    # Thái Bình
    # Hà Nam
    # Nam Định
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009897', 'Ninh Bình'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009808', 'Thanh Hoá'],
    # Nghệ An
    # Hà Tĩnh
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009827', 'Quảng Bình'],
    # Quảng Trị
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009805', 'Quảng Nam'],
    # Quảng Ngãi
    # Kon Tum
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009960', 'Gia Lai'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009974', 'Bình Định'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009823', 'Phú Yên'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010050', 'Đắk Lắk'],
    # Đắk Nông
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010067', 'Cam Ranh - Khánh Hoà'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009841', 'Nha Trang - Khánh Hoà'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010000', 'Lâm Đồng'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009796', 'Ninh Thuận'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009825', 'Bình Thuận'],
    # Bình Phước
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009931', 'Tây Ninh'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010019', 'Bình Dương'],
    # Đồng Nai
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009889', 'Bà Rịa - Vũng Tàu'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010439', 'Long An'],
    # Đồng Tháp
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010107', 'Mỹ Tho - Tiền Giang'],
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010020', 'Bến Tre'],
    # An Giang
    # Vĩnh Long
    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10011570', 'Phú Quốc - Kiên Giang']
    # Hậu Giang
    # Trà Vinh
    # Sóc Trăng
    # Bạc Liêu
    # Cà Mau
]

In [ ]:
links = [

    ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010647', 'Lào Cai'],
    # Cao Bằng
    # Lạng Sơn
    # Tuyên Quang
    # Bắc Kạn

    # Phú Thọ
    # Bắc Ninh
    # Bắc Giang
    # ['https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010278', 'Hạ Long - Quảng Ninh'],
    # Thái Bình
    # Hà Nam
    # Nam Định
    
    # Nghệ An
    # Hà Tĩnh
    # Quảng Trị
    # Quảng Ngãi
    # Kon Tum
    # Đắk Nông
    # Bình Phước
    # Đồng Nai
    # Đồng Tháp
    # An Giang
    # Vĩnh Long

    # Hậu Giang
    # Trà Vinh
    # Sóc Trăng
    # Bạc Liêu
    # Cà Mau
]

In [487]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10011570')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10011570', 'Phú Quốc - Kiên Giang')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 6.12s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 7.89s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 7.76s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 1
🔍 Đang scrape trang 2...
Xử lý bài đăng thứ 1
Lỗi khi xử lý bài đăng (lần thử 1): Message: element click intercepted: Element <div class="css-1dbjc4n r-1pi2tsx r-13qz1uu">...</div> is not clickable at point (611, 192). Other element would receive the click: <div class="css-1dbjc4n r-1k97etb r-m53kd2 r-d9fdf6 r-tvv088 r-okql6n">...</div>
  (Session info: chrome=137.0.7151.103)
Stacktrace:
0   chromedriver                        0x0000000100b5a668 cxxbridge1$str$ptr + 2723108
1   chromedriver                        0x0000000100b528d

In [484]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010020')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010020', 'Bến Tre')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 6.56s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [483]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010107')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010107', 'Mỹ Tho - Tiền Giang')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 6.62s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [482]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010439')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010439', 'Long An')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [481]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009889')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009889', 'Bà Rịa - Vũng Tàu')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 6.53s...
Xử lý bài đăng thứ 5
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [480]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010019')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010019', 'Bình Dương')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [479]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009931')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009931', 'Tây Ninh')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.82s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [478]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009825')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009825', 'Bình Thuận')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.89s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 5.07s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 7.59s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 1
🔍 Đang scrape trang 2...
Xử lý bài đăng thứ 1
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [477]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009796')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009796', 'Ninh Thuận')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.69s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [476]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010000')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010000', 'Lâm Đồng')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 7.07s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 5.74s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 7.18s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 1
🔍 Đang scrape trang 2...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.54s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 5.44s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 7.54s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 2
🔍 Đang scrape trang 3...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
✅ Đã 

In [469]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009841')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009841', 'Nha Trang - Khánh Hoà')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 7.24s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 5.68s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 6.85s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 1
🔍 Đang scrape trang 2...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 6.68s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 7.25s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 6.65s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 2
🔍 Đang scrape trang 3...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý

In [472]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010278')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010278', 'Hạ Long - Quảng Ninh')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 6.16s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 7.00s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 6.98s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 1
🔍 Đang scrape trang 2...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 7.57s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 7.25s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 5.38s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 2
🔍 Đang scrape trang 3...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý

In [466]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010067')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010067', 'Cam Ranh - Khánh Hoà')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.65s...
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [464]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010050')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010050', 'Đắk Lắk')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [463]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009823')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009823', 'Phú Yên')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.95s...
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [462]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009974')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009974', 'Bình Định')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.88s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 6.33s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [461]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009960')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009960', 'Gia Lai')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.99s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.


In [458]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009805')
time.sleep (5)
Scraping_Pages (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009805', 'Quảng Nam')
driver.quit ()

🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 6.15s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 5.70s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 7.84s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 1
🔍 Đang scrape trang 2...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 7.91s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 7.23s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 5.87s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 2
🔍 Đang scrape trang 3...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý

In [445]:
Scraping_All_URL (links)

Đang xử lý: Lào Cai (https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010002) - Lần thử 1
🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.21s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 6.71s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 5.93s...
Xử lý bài đăng thứ 15
✅ Đã scrape xong trang 1
🔍 Đang scrape trang 2...
Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 7.16s...
Xử lý bài đăng thứ 5
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.
😴 Sleeping longer for 7.88s...
Đang xử lý: Lào Cai (https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010647) - Lần thử 1
🔍 Đang scrape trang 1...
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.
😴

KeyboardInterrupt: 

In [450]:
Scraping_All_URL (links)

Đang xử lý: Lào Cai (https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010647) - Lần thử 1
🔍 Đang scrape trang 1...
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.
😴 Sleeping longer for 5.88s...
Đang xử lý: Hà Giang (https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010592) - Lần thử 1
🔍 Đang scrape trang 1...
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.
😴 Sleeping longer for 7.41s...
Đang xử lý: Yên Bái (https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010700) - Lần thử 1
🔍 Đang scrape trang 1...
Xử lý bài đăng thứ 1
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.
😴 Sleeping longer for 7.07s...
Đang xử lý: Vĩnh Phúc (https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009875) - Lần thử 1
🔍 Đang scrape trang 1...
✅ Đã tới trang cuối cùng hoặc không tìm thấy nút Next.
😴 Sleeping longer for 6.98s...
Đang xử lý: Hoà Bình (https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009919) - Lần thử 1
🔍 Đang scra

In [457]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009827')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009827', 'Quảng Bình')
driver.quit ()

Xử lý bài đăng thứ 1


In [456]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009808')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009808', 'Thanh Hoá')
driver.quit ()

Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3


In [455]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009834')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009834', 'Văn Giang - Hưng Yên')
driver.quit ()

Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3


In [454]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009875')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009875', 'Vĩnh Phúc')
driver.quit ()

Xử lý bài đăng thứ 1


In [453]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010700')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010700', 'Yên Bái')
driver.quit ()

Xử lý bài đăng thứ 1


In [452]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010592')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=30010592', 'Hà Giang')
driver.quit ()

Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 6.45s...
Xử lý bài đăng thứ 5


In [451]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009897')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009897', 'Ninh Bình')
driver.quit ()

Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 6.49s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 7.58s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 5.90s...
Xử lý bài đăng thứ 15


In [432]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009843')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009843', 'Hà Nội')
driver.quit ()

Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 7.35s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 6.74s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 7.43s...
Xử lý bài đăng thứ 15


In [433]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010083')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010083', 'Đà Nẵng')
driver.quit ()

Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 7.76s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 5.55s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 6.37s...
Xử lý bài đăng thứ 15


In [434]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009800')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10009800', 'Hải Phòng')
driver.quit ()

Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.10s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7


In [435]:
driver = Create_WebDriver ()
driver.get ('https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010311')
time.sleep (5)
Extract_Info_From_Soup (driver, 'https://www.traveloka.com/vi-vn/activities/search?st=GEO&eid=10010311', 'Huế')
driver.quit ()

Xử lý bài đăng thứ 1
Xử lý bài đăng thứ 2
Xử lý bài đăng thứ 3
Xử lý bài đăng thứ 4
😴 Sleeping longer for 5.17s...
Xử lý bài đăng thứ 5
Xử lý bài đăng thứ 6
Xử lý bài đăng thứ 7
Xử lý bài đăng thứ 8
Xử lý bài đăng thứ 9
😴 Sleeping longer for 6.24s...
Xử lý bài đăng thứ 10
Xử lý bài đăng thứ 11
Xử lý bài đăng thứ 12
Xử lý bài đăng thứ 13
Xử lý bài đăng thứ 14
😴 Sleeping longer for 5.02s...
Xử lý bài đăng thứ 15


In [488]:
# Tìm các document không có trường 'Description'
missing_docs = list(Province_URL_Activities.find({"Description": {"$exists": False}}))

print(f"🔍 Tìm thấy {len(missing_docs)} document không có trường 'Description'.")
for docs in missing_docs:
    print (docs, '\n')

🔍 Tìm thấy 0 document không có trường 'Description'.


In [489]:
unique_values = Province_URL_Activities.distinct('Search')
print(f"🔍 Có {len(unique_values)} giá trị 'Search' duy nhất.")
print (unique_values)

🔍 Có 36 giá trị 'Search' duy nhất.
['Bà Rịa - Vũng Tàu', 'Bình Dương', 'Bình Thuận', 'Bình Định', 'Bến Tre', 'Cam Ranh - Khánh Hoà', 'Cần Thơ', 'Gia Lai', 'Hoà Bình', 'Huế', 'Hà Giang', 'Hà Nội', 'Hạ Long - Quảng Ninh', 'Hải Phòng', 'Hồ Chí Minh', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng', 'Mỹ Tho - Tiền Giang', 'Nha Trang - Khánh Hoà', 'Ninh Bình', 'Ninh Thuận', 'Phú Quốc - Kiên Giang', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Sơn La', 'Thanh Hoá', 'Thái Nguyên', 'Tây Ninh', 'Văn Giang - Hưng Yên', 'Vĩnh Phúc', 'Yên Bái', 'Đà Nẵng', 'Đắk Lắk']


In [19]:
from collections import defaultdict
duplicates = defaultdict(list)

# Duyệt qua tất cả documents có đủ 3 trường
for doc in Province_URL_Activities.find({
    'Search': {'$exists': True},
    'Title': {'$exists': True},
    'Rating': {'$exists': True}
}):
    key = (doc['Search'], doc['Title'], doc['Rating'])
    duplicates[key].append(doc['_id'])

# Lọc ra các nhóm có nhiều hơn 1 document => là trùng
duplicated_groups = {k: v for k, v in duplicates.items() if len(v) > 1}

print(f"🔁 Tổng cộng có {len(duplicated_groups)} nhóm trùng lặp.")

for key, ids in duplicated_groups.items():
    print(f"\n⚠️ Trùng: Search = {key[0]}, Title = {key[1]}, Rating = {key[2]}")
    print("📄 Document IDs:", ids)



🔁 Tổng cộng có 0 nhóm trùng lặp.


# SCRAPING DETAILS

- Location : Địa chỉ chi tiết
- Time_Working : Giờ hoạt động
- Description : Mô tả hoạt động

## Collection Reviews:
- _id : Object Id (Default MongoDB)
- activity_id : Khoá ngoại tham chiếu đến Activities
- Author : Tên tác giả
- Content : Nội dung nhận xét
- Rating : Điểm đánh giá cá nhân
- Time_Review: Thời gian đánh giá
- Img_Count : Số lượng hình ảnh có trong review
- SourcePlatform : Nguồn chứa nội dung nhận xét
* MetaData:
- DateTime: Ngày giờ thu thập
- Source_Url : URL gốc

In [15]:
cursor = Province_URL_Activities.find({}, {"_id": 1, "Url": 1})

items_to_scrape = [{"_id": doc["_id"], "Url": doc["Url"]} for doc in cursor]

items_to_scrape

[{'_id': ObjectId('684a24cc1fa24a2e6b5a8e90'),
  'Url': 'https://www.traveloka.com/vi-vn/activities/vietnam/product/the-amazing-bay-water-park-tickets-7369264504118'},
 {'_id': ObjectId('684a24f71fa24a2e6b5a8e91'),
  'Url': 'https://www.traveloka.com/vi-vn/activities/vietnam/product/ho-may-park-tickets-2001384235948'},
 {'_id': ObjectId('684a25181fa24a2e6b5a8e92'),
  'Url': 'https://www.traveloka.com/vi-vn/activities/vietnam/product/suoi-tien-theme-park-tickets-6825879014353'},
 {'_id': ObjectId('684a25341fa24a2e6b5a8e93'),
  'Url': 'https://www.traveloka.com/vi-vn/activities/vietnam/product/sun-world-ba-den-mountain-4234684001188'},
 {'_id': ObjectId('684a25571fa24a2e6b5a8e94'),
  'Url': 'https://www.traveloka.com/vi-vn/activities/vietnam/product/hcmc-only-dmaris-seafood-buffet-chain-vouchers-by-giftpop-3585196162595'},
 {'_id': ObjectId('684a257e1fa24a2e6b5a8e95'),
  'Url': 'https://www.traveloka.com/vi-vn/activities/vietnam/product/ho-chi-minh-city-double-decker-bus-by-city-sightsee

In [ ]:
# def Scrape_Experience (driver):
#     wait = WebDriverWait(driver, 15)
#     DocThem = wait.until(EC.presence_of_element_located((
#         By.XPATH, "//div[@class='css-901oao r-1gle5yg r-uh8wd5 r-1b43r93 r-b88u0q r-rjixqe r-fdjqy7']"
#     )))

#     # Scroll tới phần tử
#     driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", DocThem)

#     DocThem.click()
    
#     time.sleep (5)
#     WhatYouWillEx = wait.until(EC.presence_of_element_located((
#         By.XPATH, '//div[@data-testid="ctrWhatYoullExperienceModal"]'
#     )))
#     WhatYouWillEx = WhatYouWillEx.text.strip ()

#     close_button = wait.until(EC.presence_of_element_located((
#         By.XPATH, '//div[@data-testid="btnClose"]'
#     )))
#     # Scroll vào giữa và đợi animation (nếu có)
#     driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", close_button)
#     print ('Đã scroll')
#     time.sleep(1)
#     # Bây giờ thử click bằng JS nếu click thường vẫn lỗi
#     try:
#         wait.until(EC.element_to_be_clickable((
#             By.XPATH, '//div[@data-testid="btnClose"]'
#         ))).click()
#     except:
#         print("⚠️ Dùng JS click vì Selenium click thất bại.")
#         driver.execute_script("arguments[0].click();", close_button)
        
#     locat_time_tag = driver.find_element (By.XPATH, '//div[@class="css-1dbjc4n r-eqz5dr"]')
#     location_tag = locat_time_tag.find_element (By.XPATH, './/div[@class="css-1dbjc4n r-1awozwy r-18u37iz r-zmhzs6"]')
#     new_location = location_tag.text.strip () 
    
#     return WhatYouWillEx, new_location

In [16]:
def Scrape_Experience(driver):
    wait = WebDriverWait(driver, 15)

    try:
        DocThem = wait.until(EC.presence_of_element_located((
            By.XPATH, "//div[@class='css-901oao r-1gle5yg r-uh8wd5 r-1b43r93 r-b88u0q r-rjixqe r-fdjqy7']"
        )))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", DocThem)
        DocThem.click()
    except TimeoutException:
        print("❌ Không tìm thấy nút 'Đọc thêm'. Trả về rỗng.")
        return "", ""

    time.sleep(3)

    try:
        WhatYouWillEx = wait.until(EC.presence_of_element_located((
            By.XPATH, '//div[@data-testid="ctrWhatYoullExperienceModal"]'
        ))).text.strip()
    except TimeoutException:
        print("❌ Không tìm thấy nội dung 'What You'll Experience'.")
        WhatYouWillEx = ""

    # ✅ Đóng modal
    try:
        close_button = wait.until(EC.presence_of_element_located((
            By.XPATH, '//div[@data-testid="btnClose"]'
        )))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", close_button)
        time.sleep(1)
        try:
            wait.until(EC.element_to_be_clickable((
                By.XPATH, '//div[@data-testid="btnClose"]'
            ))).click()
        except:
            print("⚠️ Dùng JS click vì Selenium click thường thất bại.")
            driver.execute_script("arguments[0].click();", close_button)
    except TimeoutException:
        print("❌ Không tìm thấy nút đóng modal.")

    # ✅ Lấy địa điểm
    try:
        locat_time_tag = driver.find_element(By.XPATH, '//div[@class="css-1dbjc4n r-eqz5dr"]')
        location_tag = locat_time_tag.find_element(By.XPATH, './/div[@class="css-1dbjc4n r-1awozwy r-18u37iz r-zmhzs6"]')
        new_location = location_tag.text.strip()
    except NoSuchElementException:
        print("❌ Không tìm thấy thẻ chứa địa điểm.")
        new_location = ""

    return WhatYouWillEx, new_location


In [20]:
# img = driver.find_elements (By.XPATH, '//div[@class="css-1dbjc4n r-bnwqim"]//img')
# for i in img:
#     print (i.get_attribute ('srcset'))

In [36]:
def get_metadata(source):
    now = datetime.now()
    metadata = {
        "SourcePlatform": source,
        "Date": now.strftime("%Y-%m-%d"),       # ví dụ: '2025-06-11'
        "Time": now.strftime("%H:%M:%S")        # ví dụ: '17:36:27'
    }
    return metadata

from bson import ObjectId
def Scraping_Reviews (driver, url, object_id):
    authors = driver.find_elements (By.XPATH, '//div[@class="css-1dbjc4n r-13awgt0 r-9aw3ui"]')

    contents = driver.find_elements (By.XPATH, '//div[@data-testid="review-list-container"]//div[@class="css-1dbjc4n r-13qz1uu"]')
        
    per_ratings = driver.find_elements (By.XPATH, '//div[@data-testid="tvat-ratingScore"]')
        
    time_reviews = driver.find_elements (By.XPATH, '//div[@data-testid="review-list-container"]//div[@class="css-901oao r-1ud240a r-uh8wd5 r-1b43r93 r-b88u0q r-1cwl3u0 r-fdjqy7"]')
        
    count_likes = driver.find_elements (By.XPATH, '//div[@data-testid="review-list-container"]//div[@class="css-901oao r-1i6uqv8 r-uh8wd5 r-1b43r93 r-b88u0q r-1cwl3u0 r-1jkjb r-fdjqy7"]')
    likes = []
    for count_like in count_likes:
        if (count_like.text.strip () == 'Đánh giá này hữu ích không?'):
            likes.append (0)
        else:
            number = int(count_like.text.split(" ")[0])
            likes.append (number)
        
    num_reviews = min(
        len(authors),
        len(contents),
        len(per_ratings),
        len(time_reviews),
        len(likes)
    )
    
    for i in range(num_reviews):
        review_data = {
            'Activity_Id': object_id,
            "Author": authors[i].text.strip(),
            "Content": contents[i].text.strip(),
            "Per_Rating": per_ratings[i].text.strip(),
            "Time_Review": time_reviews[i].text.strip(),
            "Likes": likes[i],
        }
        metadata = get_metadata (url)
        if metadata:
            review_data.update (metadata)

        # Activities_Reviews.insert_one (review_data)
    

def Check_Info(object_id):
    # Tìm document theo ObjectId
    doc = Province_URL_Activities.find_one({"_id": BsonObjectId(object_id)})
    if not doc:
        print("Không tìm thấy document với _id:", object_id)
        return

    old_location = doc.get("Location") .strip ()

    # Scrape nội dung mới
    for i in range (3):
        description, new_location = Scrape_Experience(driver)
        if description != "":
            break
        
    # So sánh location
    is_match = (old_location == new_location)

    # Cập nhật document
    # Cập nhật Description, Location và Check
    # Province_URL_Activities.update_one(
    #     {"_id": ObjectId(object_id)},
    #     {"$set": {
    #         "Description": description,
    #         "Location": new_location,
    #         "Check": is_match
    #     }}
    # )
    print(f"✅ Đã cập nhật document. Check = {is_match}")

In [37]:
def Scraping_Pages(driver, url, object_id):
    wait = WebDriverWait(driver, 15)
    page_count = 1
    
    Check_Info (object_id)
    
    while True:
        print(f"🔍 Đang scrape reviews trang thứ {page_count}...")

        # 🌀 Thử tải trang hiện tại tối đa 3 lần
        for retry_count in range(3):
            try:
                wait.until(EC.presence_of_element_located((
                    By.XPATH, '//div[@data-testid="review-list-container"]//div[@class="css-901oao r-1i6uqv8 r-uh8wd5 r-1b43r93 r-b88u0q r-1cwl3u0 r-1jkjb r-fdjqy7"]'
                )))
                break  # ✅ Thành công
            except TimeoutException:
                print(f"⏳ Không tìm thấy nội dung, thử lại ({retry_count + 1}/3)...")
                time.sleep(2)
        else:
            print("⚠️ Không thể tải nội dung sau 3 lần thử. Bỏ qua trang này.")
            break  # ❌ Dừng hẳn việc scrape (hoặc có thể dùng `continue` nếu muốn tiếp tục vòng lặp)

        # ✅ Nếu đến đây nghĩa là nội dung đã tải → scrape reviews
        Scraping_Reviews(driver, url, object_id)
        random_delay (3,7)
        
        # 👉 Cố gắng chuyển sang trang kế tiếp
        try:
            next_button = wait.until(EC.presence_of_element_located((
                By.XPATH, "//div[@data-testid='next-page-btn']"
            )))
            if next_button.get_attribute("aria-disabled") == "true":
                print("✅ Đã đến trang cuối cùng (nút Next bị vô hiệu hóa).")
                random_delay()
                break
            
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            next_button.click()
            time.sleep(5)
            page_count += 1
            
        except (NoSuchElementException, TimeoutException):
            print("✅ Không còn trang tiếp theo. Đã scrape xong.")
            random_delay ()
            break
        except StaleElementReferenceException:
            print("⚠️ Nút Next bị stale, thử lại.")
            random_delay ()
            break
        except Exception as e:
            print(f"❌ Lỗi khi xử lý trang {page_count + 1}: {e}")
            random_delay ()
            break

In [38]:
driver = Create_WebDriver ()

In [39]:
object_id = ObjectId(items_to_scrape [0]['_id'])
url = items_to_scrape [0]['Url']        # Lấy URL
driver.get (url)

In [40]:
time.sleep (15)
random_delay ()
Scraping_Pages (driver, url, object_id)
driver.quit ()

✅ Đã cập nhật document. Check = True
🔍 Đang scrape reviews trang thứ 1...
🔍 Đang scrape reviews trang thứ 2...
🔍 Đang scrape reviews trang thứ 3...
🔍 Đang scrape reviews trang thứ 4...
🔍 Đang scrape reviews trang thứ 5...
🔍 Đang scrape reviews trang thứ 6...
🔍 Đang scrape reviews trang thứ 7...
✅ Đã đến trang cuối cùng (nút Next bị vô hiệu hóa).


In [19]:
def Scrape(idx):
    def start_driver():
        return Create_WebDriver()

    driver = start_driver()
    scraped = 0

    for item in items_to_scrape[idx:]:
        object_id = ObjectId(item['_id'])  # Lấy ID dạng ObjectId
        url = item['Url']

        try:
            driver.get(url)
            time.sleep(15)
            random_delay()

            print(f'🔄 Tiến hành scrape url thứ {idx} | Object ID: {object_id}')
            Scraping_Pages(driver, url, object_id)
        except Exception as e:
            print(f"❌ Lỗi khi scrape {url}: {e}")
            # Thử reset driver nếu lỗi do connection
            driver.quit()
            driver = start_driver()
        finally:
            idx += 1
            scraped += 1

            if scraped % 5 == 0:
                print(f'📊 Đã scrape được {scraped} URL...')
                random_delay(10, 15)

            # Sau mỗi 20 URL: nghỉ lâu + reset driver
            if scraped % 20 == 0:
                print(f'😴 Nghỉ dài sau {scraped} URL...')
                random_delay(60, 120)
                driver.quit()
                driver = start_driver()

    driver.quit()
    print("✅ Scraping hoàn tất.")


In [34]:
driver = Create_WebDriver ()
idx = 776
for item in items_to_scrape[776:]:
    object_id = ObjectId(item['_id'])  # Lấy ID dưới dạng string
    url = item['Url']             # Lấy URL
    
    driver.get (url)
    time.sleep (15)
    random_delay ()
    
    print (f'🔄 Tiến hành scrape url thứ {idx} Object Id {object_id}')
    Scraping_Pages (driver, url, object_id)
    idx += 1
    
    if idx % 5 == 0:
        print(f'📊 Đã scrape được {idx} URL...')
        random_delay(10, 15)
    
    # ⏳ Nghỉ lâu hơn sau mỗi 20 URL
    if idx % 20 == 0:
        print(f'😴 Nghỉ dài sau {idx} URL đã scrape...')
        random_delay(60, 120)  # Delay dài hơn (1–2 phút, tùy bạn)
        
driver.quit ()

🔄 Tiến hành scrape url thứ 776 Object Id 684aac4c1fa24a2e6b5a9279
✅ Đã cập nhật document. Check = True
🔍 Đang scrape reviews trang thứ 1...
✅ Đã đến trang cuối cùng (nút Next bị vô hiệu hóa).
🔄 Tiến hành scrape url thứ 777 Object Id 684aac5a1fa24a2e6b5a927a
✅ Đã cập nhật document. Check = True
🔍 Đang scrape reviews trang thứ 1...
🔍 Đang scrape reviews trang thứ 2...
✅ Đã đến trang cuối cùng (nút Next bị vô hiệu hóa).
🔄 Tiến hành scrape url thứ 778 Object Id 684aac721fa24a2e6b5a927b
✅ Đã cập nhật document. Check = True
🔍 Đang scrape reviews trang thứ 1...
✅ Đã đến trang cuối cùng (nút Next bị vô hiệu hóa).
🔄 Tiến hành scrape url thứ 779 Object Id 684aac881fa24a2e6b5a927c
✅ Đã cập nhật document. Check = True
🔍 Đang scrape reviews trang thứ 1...
⏳ Không tìm thấy nội dung, thử lại (1/3)...
⏳ Không tìm thấy nội dung, thử lại (2/3)...
⏳ Không tìm thấy nội dung, thử lại (3/3)...
⚠️ Không thể tải nội dung sau 3 lần thử. Bỏ qua trang này.
📊 Đã scrape được 780 URL...
😴 Nghỉ dài sau 780 URL đã sc

In [33]:
driver.quit ()

In [21]:
Scrape (22)

🔄 Tiến hành scrape url thứ 22 | Object ID: 684a27341fa24a2e6b5a8ea6
❌ Lỗi khi scrape https://www.traveloka.com/vi-vn/activities/vietnam/product/van-gogh-and-monet-art-lighting-exhibition-ho-chi-minh-city-vietnam-6953437269294: name 'driver' is not defined


KeyboardInterrupt: 

In [58]:
driver.quit()

In [610]:
result = Scraping_Reviews (driver, 'abc.com', '2')
for r in result.values ():
    print (r)

Guest
Lên Hồ Mây mát mẻ, nếu mà cho các bạn trẻ thì bổ sung thêm trò chơi
7,0
Đánh giá cách đây 91 tuần
2
abc.com
2025-06-12
23:07:23


In [22]:
from selenium.common.exceptions import WebDriverException, TimeoutException
import requests.exceptions

try:
    driver = Create_WebDriver()
    url = items_to_scrape[1]['Url']
    print(f"🌐 Đang mở URL: {url}")
    
    driver.get("https://www.traveloka.com/vi-vn/activities/vietnam/product/van-gogh-and-monet-art-lighting-exhibition-ho-chi-minh-city-vietnam-6953437269294")
    time.sleep(5)
    print("✅ Trang đã được mở thành công.")

except (WebDriverException, TimeoutException, requests.exceptions.RequestException) as e:
    print(f"❌ Lỗi khi mở URL: {e}")
    try:
        driver.quit()
    except:
        pass
    # Bạn có thể khởi tạo lại driver ở đây nếu muốn tiếp tục
    # driver = Create_WebDriver()

except Exception as e:
    print(f"⚠️ Lỗi không xác định: {e}")
    try:
        driver.quit()
    except:
        pass


🌐 Đang mở URL: https://www.traveloka.com/vi-vn/activities/vietnam/product/ho-may-park-tickets-2001384235948
✅ Trang đã được mở thành công.


# B

In [556]:
driver.get (items_to_scrape [3]['Url'])

In [538]:
DocThem = driver.find_element(By.XPATH, "//div[@class='css-901oao r-1gle5yg r-uh8wd5 r-1b43r93 r-b88u0q r-rjixqe r-fdjqy7']")
DocThem.click()

scraping
crawling

In [1]:
import requests